In [40]:
import torch
import nltk
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

# Pre processing of training data

In [36]:

from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenize(sentence):
    return nltk.word_tokenize(sentence)

def stem(word):
    return stemmer.stem(word.lower())

def bag_of_words(tokenized_sentence, all_words):
    tokenized_sentence = [stem(w) for w in tokenized_sentence]
    
    bag = np.zeros(len(all_words), dtype=np.float32)
    for idx, w, in enumerate(all_words):
        if w in tokenized_sentence:
            bag[idx] = 1.0
    return bag

sentence = ['Hi', 'Hey', 'Is anyone there?', 'Hello', 'Hay']

words = ['Hi', 'Hey', 'I', 'Hello', 'you']
bag = bag_of_words(sentence, words)

words = ["organize", "organize", "organize"]
stemmed_words = [stem(w) for w in words]
stemmed_words

['organ', 'organ', 'organ']

# create training data

In [37]:
import json 
import numpy as np

with open('intents.json', 'r') as f:
    intents = json.load(f)
    #print(intents)
all_words = []
tags = []
xy = []
for intent in intents['intents']:
    tag = intent['tag']
    tags.append(tag)
    for pattern in intent['patterns']:
        w = tokenize(pattern)
        all_words.extend(w)
        xy.append((w, tag))

ignore_words = ['?', '!', '.', ',']
all_words = [stem(w) for w in all_words if w not in ignore_words]
all_words = sorted(set(all_words))
tags = sorted(set(tags))

x_train = []
y_train = []
for (pattern_sentence, tag) in xy:
    bag = bag_of_words(pattern_sentence, all_words)
    x_train.append(bag)
    
    label = tags.index(tag)
    y_train.append(label)
    
x_train = np.array(x_train)
y_train = np.array(y_train)

# Pytorch dataset

In [96]:
class ChatDataset():
    def __init__(self):
        self.n_samples = len(x_train)
        self.x_data = x_train
        self.y_data = y_train
        
    def __getitem__ (self, index):
        return self.x_data[index], self.y_data[index]
    
    def __len__(self):
        return self.n_samples
#hyperparameters
batch_size = 8
hidden_size = 8
output_size = len(tags)
input_size = len(x_train[0])
learning_rate = 0.001
num_epochs = 1000

dataset = ChatDataset()
train_loader = DataLoader(dataset=dataset, batch_size = batch_size, shuffle=True, num_workers=0)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')       
model = NeuralNet(input_size, hidden_size, output_size).to(device)

#loss and optimizer
criterion = nn.CrossEntropyLoss()
optimzer = torch.optim.Adam(model.parameters(), lr = learning_rate)

for epoch in range(num_epochs):
    for (words, labels) in train_loader:
        words = words.to(device)
        labels = labels.to(device)
        #forward
        outputs = model(words)
        loss = criterion(outputs, labels)
        
        #backward and optimzer step
        optimzer.zero_grad()
        loss.backward()
        optimzer.step()
        
    if (epoch +1) % 100 == 0:
        print(f'epoch {epoch+1}/{num_epochs}, loss={loss.item():.4}')
        
print(f'final loss, loss={loss.item():.4f}')

data = {
    "model_state": model.state_dict(),
    "input_size": input_size,
    "output_size": output_size,
    "hidden_size": hidden_size,
    "all_words": all_words,
    "tags": tags
}

FILE = "data.pth"
torch.save(data, FILE)
print(f'training complete. file saved to {FILE}')

epoch 100/1000, loss=0.3889
epoch 200/1000, loss=0.184
epoch 300/1000, loss=2.079
epoch 400/1000, loss=0.0008703
epoch 500/1000, loss=0.0124
epoch 600/1000, loss=2.079
epoch 700/1000, loss=2.079
epoch 800/1000, loss=2.079
epoch 900/1000, loss=2.079
epoch 1000/1000, loss=0.001684
final loss, loss=0.0017
training complete. file saved to data.pth


# Pytorch model and training

In [71]:
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.l1 = nn.Linear(input_size, hidden_size)
        self.l2 = nn.Linear(hidden_size, hidden_size)
        self.l3 = nn.Linear(hidden_size, num_classes)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        out = self.relu(out)
        out = self.l3(out)
        out = self.relu(out)
        #dont activate
        return out
    
    

# Save/load and implement chatbot

In [80]:
import random

In [97]:
with open ('intents.json', 'r') as f:
    intents = json.load(f)
    
FILE = "data.pth"
data = torch.load(FILE)

input_size = data['input_size']
hidden_size = data['hidden_size']
output_size = data['output_size']
all_words = data["all_words"]
tags = data["tags"]
model_state = data["model_state"]

model = NeuralNet(input_size, hidden_size, output_size).to(device)
model.load_state_dict(model_state)
model.eval()

NeuralNet(
  (l1): Linear(in_features=53, out_features=8, bias=True)
  (l2): Linear(in_features=8, out_features=8, bias=True)
  (l3): Linear(in_features=8, out_features=8, bias=True)
  (relu): ReLU()
)

In [98]:
bot_name = "Ultron"
print("lets chat: type 'quit' to exit")
while True:
    sentence = input('You: ')
    if sentence == 'quit':
        break
        
    sentence = tokenize(sentence)
    x = bag_of_words(sentence, all_words)
    x = x.reshape(1, x.shape[0])
    x = torch.from_numpy(x)
    
    output = model(x)
    _, predicted = torch.max(output, dim=1)
    tag = tags[predicted.item()]
    
    probs = torch.softmax(output, dim=1)
    prob = probs[0][predicted.item()]
    
    
    if prob.item() > 0.75:
        for intent in intents["intents"]:
            if tag ==intent["tag"]:
                print(f"{bot_name}: {random.choice(intent['responses'])}")
    else:
        print(f"{bot_name}: I do not understand...")

lets chat: type 'quit' to exit


You:  hi


Ultron: Hi


You:  whats your name?


Ultron: I do not understand...


...

You:  thanks


Ultron: Happy to help!


You:  Who are you?


Ultron: I'm ultron, your assistant


You:  quit
